<a href="https://colab.research.google.com/github/ty875/BeautifulSoup/blob/main/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load in necessary libraries

In [3]:
import requests #pip install requests
from bs4 import BeautifulSoup as bs #pip install beautifulSoup4


# Load our first page

In [ ]:
#Load the webpage content
r = requests.get("https://ty875.github.io/BeautifulSoup/")

#Convert to a beautiful soup object
soup = bs(r.content)

#Print out the html
print(soup.prettify())

# Start Scraping

#### find and find_all

In [ ]:
first_header = soup.find("h2")

headers = soup.find_all("h2") #a list of elements
print(headers)

In [ ]:
#Pass in a list of elements to look for
first_header = soup.find(["h2","h1"])

headers = soup.find_all(["h1","h2"])
headers

In [ ]:
#pass in attributes to the find/find_all
paragraph = soup.find_all("p", attrs={"id": "paragraph-id"})
paragraph

In [ ]:
#nesting find/find_all calls
body = soup.find('body')
div = body.find('div')
header = div.find('h1')
header

In [ ]:
#search for specific string in find/find_all calls
import re

paragraphs = soup.find_all(lambda tag: "p", string=re.compile("Some"))
paragraphs

#headers = soup.find_all("h2", string = re.compile("(H|h)eader"))
#headers

# select(CSS selector)

In [ ]:
content = soup.select("div p") #p tag only inside div
content

In [ ]:
paragraphs = soup.select("h2 ~ p") #p tag strictly after h2 tag
paragraphs

In [ ]:
bold_text = soup.select("p#paragraph-id b")
bold_text

In [ ]:
paragraphs = soup.select("body > p") #> direct descendant of body
#print(paragraphs)

for paragraph in paragraphs:
  print(paragraph.select("i"))

In [ ]:
#Grab by element with specific propert
soup.select("[align = middle]")

# Get different properties of he HTML

In [ ]:
#string
header = soup.find("h2")
header.string

#IF multiple child elements use get_text()
div = soup.find("div")
print(div.prettify())
print(div.get_text())

In [ ]:
#Get a specific property from an element
link = soup.find("a")
link['href']

paragraphs = soup.select("p#paragraph-id")
paragraphs[0]['id']

'paragraph-id'

#Code navigation

In [ ]:
#Path Syntax
soup.body.div.h1.string

'This is a header'

In [ ]:
#Terms: parent, sibling child
soup.body.find('div').find_next_siblings()

# Exercises
Go to https://www.google.com/

In [ ]:
from requests.api import request
#Load the page
r = requests.get("https://www.google.com/")
web = bs(r.content)
print(web.prettify())

# Grab all of the links
In at least 2 different way:

In [ ]:
#web.body.select("links with class gb1")
links = web.body.select("a.gb1") #id .class
actual_links = [link['href']for link in links]
actual_links

# Scrape the table

In [ ]:
#scrape the table into pandas dataframe
import pandas as pd

table = soup.select(".Cats_Table")[0]
columns = table.find("thead").find_all("th")
column_names = [c.string for c in columns]

table_rows = table.find("tbody").find_all("tr")
l=[]
for tr in table_rows:
  td = tr.find_all('td')
  row = [tr.get_text() for tr in td]
  l.append(row)

df = pd.DataFrame(l, columns = column_names)
df.head()
df['Trait']
df.loc[df['Breed'] != "Maine Coon"]

# Grab all the Breeds that has the trait: friendly

In [ ]:
import re

Cats = soup.select('.Cats_Table')
Friendly_Cats = [cat.find_all(string = re.compile("friendly")) for cat in Cats]
Friendly_Cats

Goofy_Cats = [cat.find(string = re.compile("goofy")) for cat in Cats]
Goofy_Cats = [cat.find_parent().get_text() for cat in Goofy_Cats if cat]
Goofy_Cats

# Download an image

In [5]:
import requests

url = "https://ty875.github.io/BeautifulSoup/"

image = soup.select("img")
image_url = image[0]['src']

image_data = requests.get(image_url).content
#download the image into the google collab directory
#can be accessed by going to the file folder to the left
#To download it on local machines, use terminal
with open('Cat.jpg', 'wb') as handler:
    handler.write(image_data)

# Solving the mystery challenge

In [ ]:
Files = soup.select("li a")
relative_files = [f['href'] for f in Files]

url = "https://ty875.github.io/BeautifulSoup/"
for f in relative_files:
  full_url = url + f
  page = requests.get(full_url)
  bs_page = bs(page.content)
  secret_word = bs_page.find("p", attrs={"id":"secret-word"})
  print(secret_word.string)